In [1]:
import json
import pandas as pd

In [3]:
import os
import glob

# Base directory containing all folders
base_directory = "./generated_answers/gpt-4o-2024-05-13/test_must_why"

# Use glob to find all JSONL files recursively
jsonl_files = glob.glob(os.path.join(base_directory, "**", "*.jsonl"), recursive=True)

# Example: Print all found JSONL file paths
#print(jsonl_files)

In [27]:
# List of JSONL files
files = [['dependent_real_after_basic_binary_explain.jsonl', 'dependent_real_before_basic_binary_explain.jsonl'], ['nondependent_real_after_basic_binary_explain.jsonl', 'nondependent_real_before_basic_binary_explain.jsonl']]
files_to_use = files[0]

In [28]:
#Filter the list of found files based on your selection
selected_files = [file for file in jsonl_files if os.path.basename(file) in files_to_use]

# Example: Print selected file paths
print(selected_files)

['./generated_answers/gpt-4o-2024-05-13/test_must_why/dependent_real_after_basic_binary_explain.jsonl', './generated_answers/gpt-4o-2024-05-13/test_must_why/dependent_real_before_basic_binary_explain.jsonl']


In [29]:
data = []

for file_path in selected_files:
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

In [30]:
df

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,model_answer
0,0,spicy-tomato-anchovy-pasta,0,[Heat 6 tablespoons olive oil in a large fryin...,dependent_real_after,"[7, 9]",Must Step 10 happen after Step 8?,Why must Step 10 happen after Step 8?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
1,1,one-pot irish stew,1,"[Place the lamb in a large pot, cover with wat...",dependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why must Step 8 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
2,2,easy strawberry tart,2,"[Mix together the sugar, water, jam and cornfl...",dependent_real_after,"[5, 6]",Must Step 7 happen after Step 6?,Why must Step 7 happen after Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
3,3,almond and apple cake,3,"[Preheat the oven to 170 C / Gas mark 3., Brus...",dependent_real_after,"[5, 7]",Must Step 8 happen after Step 6?,Why must Step 8 happen after Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
4,4,chocolate-chocolate-cake,4,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_after,"[0, 20]",Must Step 21 happen after Step 1?,Why must Step 21 happen after Step 1?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1415,705,rouille,705,"[In a mortar, crush the garlic to a fine mash....",dependent_real_before,"[1, 2]",Must Step 2 happen before Step 3?,Why must Step 2 happen before Step 3?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
1416,706,lemon-zested-strawberry-shortcakes,706,"[In a small bowl, combine strawberries and sug...",dependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
1417,707,chocolate-chocolate-cake,707,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_before,"[1, 12]",Must Step 2 happen before Step 13?,Why must Step 2 happen before Step 13?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
1418,708,feta-and-sundried-tomato-spinach-rolls,708,"[Preheat grill., Mix together cream cheese, fe...",dependent_real_before,"[5, 8]",Must Step 6 happen before Step 9?,Why must Step 6 happen before Step 9?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."


In [31]:
import ast

In [32]:
def clean_model_answer(text):
    try:
        # Remove newline characters and extra backslashes
        text = text.replace('\n', '').replace('\\', '').replace("json", "").strip("```")
        # Ensure the JSON string is properly closed
        if not text.endswith('}'):
            text += '}'
        # Parse the string as JSON
        data = json.loads(text)
        # Extract the 'binary_answer' and 'why_answer'
        binary_answer = data.get('binary_answer', '')
        why_answer = data.get('why_answer', '')
        return pd.Series([binary_answer, why_answer])
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Problematic text: {text}")
        return pd.Series(['', ''])
    except Exception as e:
        print(f"Unexpected error: {e}")
        print(f"Problematic text: {text}")
        return pd.Series(['', ''])

In [33]:
# Apply the cleaning function to the 'model_answer' column
df[['binary_answer', 'why_answer']] = df['model_answer'].apply(clean_model_answer)

In [34]:
df

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,model_answer,binary_answer,why_answer
0,0,spicy-tomato-anchovy-pasta,0,[Heat 6 tablespoons olive oil in a large fryin...,dependent_real_after,"[7, 9]",Must Step 10 happen after Step 8?,Why must Step 10 happen after Step 8?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,"Step 10 involves serving the dish, which can o..."
1,1,one-pot irish stew,1,"[Place the lamb in a large pot, cover with wat...",dependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why must Step 8 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,"Step 8 involves serving the stew, which can on..."
2,2,easy strawberry tart,2,"[Mix together the sugar, water, jam and cornfl...",dependent_real_after,"[5, 6]",Must Step 7 happen after Step 6?,Why must Step 7 happen after Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,The tart needs to be well set before adding th...
3,3,almond and apple cake,3,"[Preheat the oven to 170 C / Gas mark 3., Brus...",dependent_real_after,"[5, 7]",Must Step 8 happen after Step 6?,Why must Step 8 happen after Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 8 involves combining the flour and milk w...
4,4,chocolate-chocolate-cake,4,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_after,"[0, 20]",Must Step 21 happen after Step 1?,Why must Step 21 happen after Step 1?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 21 involves cooling the cake after it has...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,705,rouille,705,"[In a mortar, crush the garlic to a fine mash....",dependent_real_before,"[1, 2]",Must Step 2 happen before Step 3?,Why must Step 2 happen before Step 3?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 2 must happen before Step 3 because the g...
1416,706,lemon-zested-strawberry-shortcakes,706,"[In a small bowl, combine strawberries and sug...",dependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 2 involves combining dry ingredients whic...
1417,707,chocolate-chocolate-cake,707,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_before,"[1, 12]",Must Step 2 happen before Step 13?,Why must Step 2 happen before Step 13?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 2 involves preparing the chocolate mixtur...
1418,708,feta-and-sundried-tomato-spinach-rolls,708,"[Preheat grill., Mix together cream cheese, fe...",dependent_real_before,"[5, 8]",Must Step 6 happen before Step 9?,Why must Step 6 happen before Step 9?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Brushing with olive oil is necessary before gr...


In [35]:
df_copy = df.copy()

In [36]:
df_copy = df_copy.drop('model_answer', axis=1)

In [37]:
df_copy

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer
0,0,spicy-tomato-anchovy-pasta,0,[Heat 6 tablespoons olive oil in a large fryin...,dependent_real_after,"[7, 9]",Must Step 10 happen after Step 8?,Why must Step 10 happen after Step 8?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 10 involves serving the dish, which can o..."
1,1,one-pot irish stew,1,"[Place the lamb in a large pot, cover with wat...",dependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why must Step 8 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 8 involves serving the stew, which can on..."
2,2,easy strawberry tart,2,"[Mix together the sugar, water, jam and cornfl...",dependent_real_after,"[5, 6]",Must Step 7 happen after Step 6?,Why must Step 7 happen after Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,The tart needs to be well set before adding th...
3,3,almond and apple cake,3,"[Preheat the oven to 170 C / Gas mark 3., Brus...",dependent_real_after,"[5, 7]",Must Step 8 happen after Step 6?,Why must Step 8 happen after Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 8 involves combining the flour and milk w...
4,4,chocolate-chocolate-cake,4,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_after,"[0, 20]",Must Step 21 happen after Step 1?,Why must Step 21 happen after Step 1?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 21 involves cooling the cake after it has...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,705,rouille,705,"[In a mortar, crush the garlic to a fine mash....",dependent_real_before,"[1, 2]",Must Step 2 happen before Step 3?,Why must Step 2 happen before Step 3?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 2 must happen before Step 3 because the g...
1416,706,lemon-zested-strawberry-shortcakes,706,"[In a small bowl, combine strawberries and sug...",dependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 2 involves combining dry ingredients whic...
1417,707,chocolate-chocolate-cake,707,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_before,"[1, 12]",Must Step 2 happen before Step 13?,Why must Step 2 happen before Step 13?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 2 involves preparing the chocolate mixtur...
1418,708,feta-and-sundried-tomato-spinach-rolls,708,"[Preheat grill., Mix together cream cheese, fe...",dependent_real_before,"[5, 8]",Must Step 6 happen before Step 9?,Why must Step 6 happen before Step 9?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Brushing with olive oil is necessary before gr...


In [38]:
counts = df_copy['binary_answer'].value_counts()
num_yes = counts['yes']
num_no = counts['no']

In [39]:
num_yes

1215

In [40]:
num_no

205

In [41]:
df_no = df_copy[df_copy['binary_answer'] == 'no']

In [42]:
df_no

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer
5,5,fusilli-and-meatballs,5,[Coat a large saucepan with cooking spray or a...,dependent_real_after,"[4, 16]",Must Step 17 happen after Step 5?,Why must Step 17 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 17 must happen after Step 16, not necessa..."
6,6,spiced butternut squash soup,6,"[Preheat the oven to 190 C / Gas mark 6., Pour...",dependent_real_after,"[1, 14]",Must Step 15 happen after Step 2?,Why must Step 15 happen after Step 2?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 15 involves seasoning and adding liquids ...
7,7,spiced butternut squash soup,7,"[Preheat the oven to 190 C / Gas mark 6., Pour...",dependent_real_after,"[3, 14]",Must Step 15 happen after Step 4?,Why must Step 15 happen after Step 4?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 15 involves seasoning and adding liquids ...
8,8,vegetable-terrine,8,[Add the gelatin in a bowl of cold water until...,dependent_real_after,"[4, 10]",Must Step 11 happen after Step 5?,Why must Step 11 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 11 involves arranging the vegetables in t...
14,14,fusilli-and-meatballs,14,[Coat a large saucepan with cooking spray or a...,dependent_real_after,"[14, 19]",Must Step 20 happen after Step 15?,Why must Step 20 happen after Step 15?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 20 can happen after Step 19 as it involve...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,331,cranachan,331,"[Preheat the oven to 180 C / Gas mark 4., Spre...",dependent_real_before,"[3, 4]",Must Step 4 happen before Step 5?,Why must Step 4 happen before Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 4 and Step 5 can happen in any order as b...
1052,342,one-pot irish stew,342,"[Place the lamb in a large pot, cover with wat...",dependent_real_before,"[5, 6]",Must Step 6 happen before Step 7?,Why must Step 6 happen before Step 7?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 6 and Step 7 can be interchanged as seaso...
1158,448,fusilli-and-meatballs,448,[Coat a large saucepan with cooking spray or a...,dependent_real_before,"[0, 12]",Must Step 1 happen before Step 13?,Why must Step 1 happen before Step 13?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 1 is related to preparing the saucepan fo...
1244,534,chocolate-pancake-batter,534,"[Sift the flour and cocoa powder into a bowl.,...",dependent_real_before,"[5, 8]",Must Step 6 happen before Step 9?,Why must Step 6 happen before Step 9?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 6 must happen before Step 7, not necessar..."


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [50]:
why_answer = df_no['why_answer'].tolist()

In [51]:
# Create a bag of words representation
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(why_answer)

In [52]:
#Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X)

KMeans(n_clusters=4)

In [53]:
# Add cluster labels
df_no['cluster'] = kmeans.labels_

/var/folders/j9/5zmzdnn93jj4z_0j5tw59c540000gn/T/ipykernel_39006/3760491917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no['cluster'] = kmeans.labels_


In [54]:
df_no

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer,cluster
5,5,fusilli-and-meatballs,5,[Coat a large saucepan with cooking spray or a...,dependent_real_after,"[4, 16]",Must Step 17 happen after Step 5?,Why must Step 17 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 17 must happen after Step 16, not necessa...",2
6,6,spiced butternut squash soup,6,"[Preheat the oven to 190 C / Gas mark 6., Pour...",dependent_real_after,"[1, 14]",Must Step 15 happen after Step 2?,Why must Step 15 happen after Step 2?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 15 involves seasoning and adding liquids ...,0
7,7,spiced butternut squash soup,7,"[Preheat the oven to 190 C / Gas mark 6., Pour...",dependent_real_after,"[3, 14]",Must Step 15 happen after Step 4?,Why must Step 15 happen after Step 4?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 15 involves seasoning and adding liquids ...,0
8,8,vegetable-terrine,8,[Add the gelatin in a bowl of cold water until...,dependent_real_after,"[4, 10]",Must Step 11 happen after Step 5?,Why must Step 11 happen after Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 11 involves arranging the vegetables in t...,0
14,14,fusilli-and-meatballs,14,[Coat a large saucepan with cooking spray or a...,dependent_real_after,"[14, 19]",Must Step 20 happen after Step 15?,Why must Step 20 happen after Step 15?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 20 can happen after Step 19 as it involve...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,331,cranachan,331,"[Preheat the oven to 180 C / Gas mark 4., Spre...",dependent_real_before,"[3, 4]",Must Step 4 happen before Step 5?,Why must Step 4 happen before Step 5?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 4 and Step 5 can happen in any order as b...,0
1052,342,one-pot irish stew,342,"[Place the lamb in a large pot, cover with wat...",dependent_real_before,"[5, 6]",Must Step 6 happen before Step 7?,Why must Step 6 happen before Step 7?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 6 and Step 7 can be interchanged as seaso...,0
1158,448,fusilli-and-meatballs,448,[Coat a large saucepan with cooking spray or a...,dependent_real_before,"[0, 12]",Must Step 1 happen before Step 13?,Why must Step 1 happen before Step 13?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 1 is related to preparing the saucepan fo...,0
1244,534,chocolate-pancake-batter,534,"[Sift the flour and cocoa powder into a bowl.,...",dependent_real_before,"[5, 8]",Must Step 6 happen before Step 9?,Why must Step 6 happen before Step 9?,gpt-4o-2024-05-13,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 6 must happen before Step 7, not necessar...",2


In [55]:
df_no.to_csv('dependent_real_basic_binary_explain_no.csv', index=False)

In [57]:
#rows_with_label = df_yes[df_yes['cluster'] == 0]